<a href="https://colab.research.google.com/github/JoaoADuarte/ChatBot/blob/main/C%C3%B3pia_de_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium transformers yake google-auth
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!pip install selenium
!pip install transformers datasets peft
!pip install evaluate
!pip install rouge_score
!pip install numpy
!pip install datasets
!pip install torch
!pip install NLTK
!pip install --upgrade transformers
!pip install gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 34.7 MB/s eta 0:00:00
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import transformers
print(f"Transformers versão instalada: {transformers.__version__}")

Transformers versão instalada: 4.51.3


In [ ]:
if 'google.colab' in str(get_ipython()):
    try:
        from google.colab import userdata
        hf_token = userdata.get('HF_TOKEN')
    except ImportError:
        print("Tentando usar google.colab.userdata, mas não encontrado.")
        hf_token = None # Ou defina um valor padrão ou lance um erro


In [ ]:
import os
import time
import sqlite3
import torch # Necessário para o modelo T5 de resumo
import yake # Necessário para extrair tags
import random # Necessário para a pausa aleatória no scraping
import re # Necessário para a função extrair_conteudo_principal
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from transformers import T5ForConditionalGeneration, T5Tokenizer # Necessário para o modelo T5 de resumo
from huggingface_hub import login, HfFolder # Mantido caso o modelo T5 precise ou para consistência

# --- Configuração Inicial ---
hf_token = os.getenv("HF_TOKEN")

if hf_token:
    print("🔑 Token Hugging Face encontrado. Fazendo login...")
    login(token=hf_token)
    # Salva o token para uso futuro na sessão/máquina
    HfFolder.save_token(hf_token)
else:
    print("⚠️ Token Hugging Face (HF_TOKEN) não encontrado nas variáveis de ambiente.")
    print("   O script pode falhar ao interagir com modelos privados ou ao fazer uploads.")
    # login() # Isso pedirá o token interativamente no terminal

# Caminho do banco de dados
default_db_name = "chatbot_jardinagem_novo2.db"
if os.path.exists("/content/drive/MyDrive"):
    drive_folder = "/content/drive/MyDrive/chatbot_jardinagem"
    if not os.path.exists(drive_folder):
         os.makedirs(drive_folder) # Cria a pasta se não existir
    db_path = os.path.join(drive_folder, default_db_name)
else:
    # Caso contrário, usa o diretório local
    db_path = default_db_name

print(f"💾 Usando banco de dados em: {db_path}")

# YAKE Keyword Extractor
kw_extrator = yake.KeywordExtractor(lan="pt", n=1, top=10) # Extrai 10 palavras-chave de 1 termo

# Selenium WebDriver Options
chrome_options = Options()
chrome_options.add_argument("--headless") # Roda sem abrir janela do navegador
chrome_options.add_argument("--no-sandbox") # Necessário em alguns ambientes Linux/Docker
chrome_options.add_argument("--disable-dev-shm-usage") # Evita problemas de memória compartilhada
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36") # Define um user-agent comum

# Tenta inicializar o driver
try:
    driver = webdriver.Chrome(options=chrome_options)
    print("🚀 WebDriver do Selenium inicializado.")
except Exception as e:
    print(f"❌ Falha ao inicializar o WebDriver: {e}")
    print("   Verifique se o ChromeDriver está instalado e no PATH do sistema,")
    print("   ou se o pacote 'selenium' e 'webdriver-manager' (opcional) estão instalados.")
    exit() # Encerra o script se o driver não puder ser iniciado

# Modelo T5 para Resumo (usado no scraping para a coluna conteudo_resumido)
print("🧠 Carregando modelo T5-base para resumo...")
try:
    tokenizer_resumo = T5Tokenizer.from_pretrained("t5-base")
    model_resumo = T5ForConditionalGeneration.from_pretrained("t5-base")
    print("✅ Modelo T5-base para resumo carregado.")
except Exception as e:
    print(f"❌ Falha ao carregar modelo T5-base: {e}")
    print("   Verifique sua conexão com a internet e se a biblioteca 'transformers' e 'torch'/'tensorflow' estão instaladas.")
    # Você pode decidir se quer continuar sem resumo ou encerrar
    # exit() # Descomente para encerrar se o resumo for essencial

# --- Banco de Dados ---
try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS artigos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        fonte TEXT,
        titulo TEXT UNIQUE, -- Adicionado UNIQUE para garantir títulos únicos
        conteudo_resumido TEXT, -- Conteúdo gerado pelo T5-base
        conteudo_completo TEXT, -- Conteúdo 'bruto' extraído
        tags TEXT, -- Tags geradas pelo YAKE
        data_extracao TEXT
    );""")
    # Adiciona índice no título para buscas mais rápidas
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_titulo ON artigos (titulo);")
    conn.commit()
    print("📦 Tabela 'artigos' no banco de dados verificada/criada.")
except sqlite3.Error as e:
    print(f"❌ Erro crítico ao conectar ou configurar o banco de dados: {e}")
    exit() # Encerra se não puder usar o banco

# Função de inserção
def inserir_dados(fonte, titulo, conteudo_resumido, conteudo_completo, tags):
    """Insere os dados coletados no banco de dados, evitando duplicatas de título."""
    # Verifica se o conteúdo completo tem um tamanho mínimo
    min_len_conteudo = 150 # Define um tamanho mínimo para o conteúdo completo
    if not conteudo_completo or len(conteudo_completo.strip()) < min_len_conteudo:
        print(f"⚠️ Conteúdo completo muito curto ({len(conteudo_completo.strip())} chars, mínimo: {min_len_conteudo}). Ignorado: {titulo[:60]}...")
        return False
    if not titulo:
        print("⚠️ Título vazio. Ignorado.")
        return False

    try:
        cursor.execute("""INSERT INTO artigos (fonte, titulo, conteudo_resumido, conteudo_completo, tags, data_extracao)
                         VALUES (?, ?, ?, ?, ?, ?)""",
                       (fonte, titulo.strip(), conteudo_resumido.strip(), conteudo_completo.strip(), tags, datetime.now().isoformat()))
        conn.commit()
        print(f"✅ Inserido: {titulo[:60]}...")
        return True
    except sqlite3.IntegrityError: # Erro específico para violação de UNIQUE (título duplicado)
        print(f"⚠️ Título já existente. Ignorado: {titulo[:60]}...")
        return False
    except sqlite3.Error as e:
        print(f"❌ Erro ao inserir '{titulo[:60]}...': {e}")
        return False

# --- Funções Auxiliares (mantidas como estavam) ---

def resumir_texto(texto, max_len=512, summary_max_len=150, summary_min_len=50):
    """Gera um resumo do texto usando o modelo T5-base carregado."""
    if not texto or 'model_resumo' not in globals() or 'tokenizer_resumo' not in globals():
        print("⚠️ Resumo não pode ser gerado (modelo não carregado ou texto vazio).")
        return ""
    try:
        input_text = f"summarize: {texto}"
        inputs = tokenizer_resumo(input_text, return_tensors="pt", max_length=max_len, truncation=True)
        with torch.no_grad(): # Importante para inferência
            summary_ids = model_resumo.generate(
                inputs["input_ids"],
                max_length=summary_max_len,
                min_length=summary_min_len,
                num_beams=4, # Beam search para melhor qualidade
                early_stopping=True # Para de gerar se o modelo achar que terminou
            )
        return tokenizer_resumo.decode(summary_ids[0], skip_special_tokens=True)
    except Exception as e:
        print(f"⚠️ Erro ao resumir texto: {e}")
        return "" # Retorna vazio em caso de erro

def encontrar_titulo_com_wait(driver, timeout=10):
    """Tenta encontrar o título da página usando uma lista de seletores CSS comuns."""
    seletores_titulo = [
        "h1", ".entry-title", ".post-title", "#page-title", ".tdb-title-text",
        ".post-header h1.title", "header h1", ".headline", ".article-title"
    ]
    for seletor in seletores_titulo:
        try:
            titulo_elemento = WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, seletor))
            )
            texto_titulo = titulo_elemento.text.strip()
            # Tenta pegar o atributo 'textContent' também, pode funcionar melhor em alguns casos
            if not texto_titulo:
                 texto_titulo = titulo_elemento.get_attribute('textContent').strip()

            if texto_titulo:
                print(f"✓ Título encontrado com seletor '{seletor}'.")
                return texto_titulo
        except TimeoutException:
            continue # Tenta o próximo seletor
        except Exception as e:
            print(f"🔍 Erro ao tentar seletor de título '{seletor}': {e}")
            continue
    print("⚠️ Não foi possível encontrar um título com os seletores padrão.")
    # Como último recurso, tenta pegar o <title> da página
    try:
        page_title = driver.title.strip()
        if page_title:
             print("✓ Usando <title> da página como fallback.")
             return page_title
        else:
             return None
    except Exception:
        return None

def extrair_conteudo_principal(driver, timeout=10):
    """Tenta extrair parágrafos do contêiner principal do artigo."""
    seletores_conteudo = [
        "article .entry-content", "article .post-content", "article",
        ".td-post-content", ".pf-content", ".content", ".post-body", ".materia-conteudo",
        "#content", "#main", "#main-content", "#articleBody", ".single-post-content" # Adicionado mais um comum
    ]
    container_principal = None
    for seletor in seletores_conteudo:
        try:
            container_principal = WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, seletor))
            )
            print(f"🔎 Encontrado container principal com seletor: '{seletor}'")
            break # Usa o primeiro que encontrar
        except TimeoutException:
            continue
        except Exception as e:
            print(f"🔍 Erro ao tentar seletor de conteúdo '{seletor}': {e}")
            continue

    paragrafos_texto = []
    if container_principal:
        # Prioridade: Encontra parágrafos <p> DENTRO do container principal
        paragrafos = container_principal.find_elements(By.TAG_NAME, "p")
        if paragrafos:
             paragrafos_texto = [p.text.strip() for p in paragrafos if p.text and len(p.text.strip()) > 20] # Filtra curtos/vazios
             print(f"  -> Extraídos {len(paragrafos_texto)} parágrafos (<p>) do container.")
        else:
             # Se não achar <p>, tenta pegar todo texto visível do container
             print("⚠️ Não foram encontrados <p> no container, pegando texto geral do container.")
             texto_completo_container = container_principal.text.strip()
             # Quebra em linhas como aproximação de parágrafos, filtrando linhas vazias
             paragrafos_texto = [linha for linha in texto_completo_container.split('\n') if linha.strip()]
             print(f"  -> Extraídas {len(paragrafos_texto)} linhas de texto do container.")

    else:
        # Fallback: Se NENHUM container principal for encontrado, pega todos os <p> da página
        print("⚠️ Container principal não encontrado. Usando fallback: todos os <p> da página.")
        try:
            paragrafos = WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, "p"))
            )
            paragrafos_texto = [p.text.strip() for p in paragrafos if p.text and len(p.text.strip()) > 20]
            print(f"  -> Extraídos {len(paragrafos_texto)} parágrafos (<p>) de toda a página (fallback).")
        except TimeoutException:
            print("⚠️ Tempo esgotado ao buscar todos os <p> como fallback.")
            return "" # Retorna vazio se o fallback falhar
        except Exception as e:
            print(f"⚠️ Erro ao buscar todos os <p> como fallback: {e}")
            return ""

    # Junta os parágrafos filtrados
    conteudo_final = "\n\n".join(paragrafos_texto) # Usa duas quebras de linha para separar parágrafos

    # Limpeza básica adicional
    # Remove linhas começando com variações de "leia mais", "veja também", etc.
    conteudo_final = re.sub(r'^\s*(leia mais|veja também|relacionado|fonte[s]?|crédito[s]?|imagem[s]?)\b.*$', '', conteudo_final, flags=re.IGNORECASE | re.MULTILINE)
    # Remove excesso de linhas em branco (mais de 2 seguidas)
    conteudo_final = re.sub(r'\n{3,}', '\n\n', conteudo_final)

    return conteudo_final.strip()

# --- Scraping Loop ---
urls = [
    "https://armazemagricola.pt/como-plantar-tomates/",
    "https://www.terra.com.br/vida-e-estilo/casa-e-decoracao/como-plantar-tomate-em-casa,f0a08517c124e92dc78d5c52650375a8hioufx0h.html",
    "https://eos.com/pt/blog/como-plantar-tomates/",
    "https://ciclovivo.com.br/mao-na-massa/horta/como-plantar-tomate-organico-em-casa/",
    "https://global.tramontina.com/blog/faca-voce-mesmo/como-plantar-tomate-em-casa",
    "https://www.cpt.com.br/artigos/horta-como-plantar-tomates-solanum-lycopersicum",
    "https://www.jardineiro.net/como-plantar-tomate-os-segredos-do-plantio-ate-a-colheita.html",
    "https://www.petz.com.br/blog/plantas/como-plantar-tomate/",
    "https://jardimdasideias.com.br/saiba-quais-as-melhores-hortalicas-para-cultivar-em-fevereiro/",
    "https://jardimdasideias.com.br/plantas-ideais-para-um-jardim-de-baixa-manutencao/",
    "https://jardimdasideias.com.br/guia-pratico-para-iniciantes-na-jardinagem-como-comecar-e-cuidar-do-seu-jardim/",
    "https://jardimdasideias.com.br/dicas-praticas-para-transformar-o-seu-jardim/",
    "https://www.jardineiro.net/40-plantas-floridas-para-canteiros-no-sol.html",
    "https://www.jardineiro.net/40-plantas-com-flor-vermelha-para-um-jardim-encantador.html",
    "https://www.jardineiro.net/8-causas-de-folhas-com-pontas-secas-e-queimadas.html",
    "https://www.jardineiro.net/suculentas-o-guia-completo-para-iniciantes.html",
    "https://www.jardineiro.net/classe/plantas-flutuantes",
    "https://www.jardineiro.net/introducao-ao-bonsai.html",
    "https://www.jardineiro.net/a-arte-japonesa-na-admiracao-da-natureza.html",
    "https://www.jardineiro.net/pragas/percevejo-do-sul-blissus-insularis.html",
    "https://www.jardineiro.net/pragas/acaros.html",
    "https://www.jardineiro.net/pragas/moscas-dos-fungos-fungus-gnats.html",
    "https://www.jardineiro.net/pragas/antracnose.html",
    "https://www.jardineiro.net/pragas/cupim-de-grama.html"
]
urls = sorted(list(set(urls))) # Remove duplicatas e ordena para consistência

print(f"\n--- Iniciando Coleta de {len(urls)} URLs ---")
artigos_inseridos_total = 0
erros_coleta = 0

for i, url in enumerate(urls):
    print(f"\n[{i+1}/{len(urls)}] 🌐 Coletando de: {url}")
    titulo = None # Reseta o título para cada URL
    try:
        driver.get(url)
        # Pequena espera para elementos dinâmicos iniciais carregarem
        titulo = encontrar_titulo_com_wait(driver)
        if not titulo:
            print(f"⚠️ Não foi possível encontrar um título adequado em {url}. Pulando.")
            erros_coleta += 1
            continue

        # Extrai o conteúdo principal
        conteudo_completo = extrair_conteudo_principal(driver)

        # Verifica o conteúdo completo ANTES de tentar resumir ou extrair tags
        if not conteudo_completo or len(conteudo_completo) < 150: # Usa o mesmo limite da função de inserir
            print(f"⚠️ Conteúdo completo insuficiente ({len(conteudo_completo)} chars) após extração para '{titulo[:60]}...'. Pulando.")
            erros_coleta += 1
            continue

        # Gera resumo a partir do conteúdo completo (se o modelo estiver carregado)
        resumo = resumir_texto(conteudo_completo)
        if not resumo:
            print("   (Falha ao gerar resumo, continuando sem ele)")
            resumo = "" # Garante que não é None para o banco

        # Extrai tags do conteúdo completo
        keywords = kw_extrator.extract_keywords(conteudo_completo)
        tags = ", ".join([kw for kw, score in keywords])
        print(f"   🏷️ Tags extraídas: {tags[:100]}...") # Mostra algumas tags

        # Insere os dados no banco
        if inserir_dados(url, titulo, resumo, conteudo_completo, tags):
            artigos_inseridos_total += 1
        else:
            erros_coleta +=1 # Conta como erro se não inseriu

        # Pausa leve e aleatória para não sobrecarregar os servidores
        tempo_pausa = random.uniform(1.5, 3.5)
        print(f"   ⏸️ Pausando por {tempo_pausa:.1f} segundos...")
        time.sleep(tempo_pausa)

    except TimeoutException as e:
        print(f"⚠️ Tempo de espera excedido ao carregar/interagir com a página {url}: {e}")
        erros_coleta += 1
    except NoSuchElementException as e:
         print(f"⚠️ Elemento não encontrado durante o processamento de {url}: {e}")
         erros_coleta += 1
    except Exception as e:
        print(f"❌ Erro inesperado ao processar {url}: {e}")
        erros_coleta += 1

# --- Finalização do Scraping ---
print("\n--- Coleta Finalizada ---")
if 'driver' in locals() and driver:
    try:
        driver.quit()
        print("🚗 WebDriver do Selenium finalizado.")
    except Exception as e:
        print(f"⚠️ Erro ao finalizar o WebDriver: {e}")

# Fecha a conexão com o banco de dados
if 'conn' in locals() and conn:
    try:
        conn.close()
        print("🚪 Conexão com o banco de dados fechada.")
    except Exception as e:
        print(f"⚠️ Erro ao fechar a conexão com o banco: {e}")

print(f"\n--- Resumo da Execução ---")
print(f"✅ Artigos inseridos com sucesso: {artigos_inseridos_total}")
print(f"❌ URLs processadas com erro ou ignoradas: {erros_coleta}")
print(f"💾 Banco de dados atualizado em: {db_path}")
print("\n🎉 Script de scraping finalizado.")

⚠️ Token Hugging Face (HF_TOKEN) não encontrado nas variáveis de ambiente ou segredos do Colab.
   O script pode falhar ao interagir com modelos privados ou ao fazer uploads.
💾 Usando banco de dados em: /content/drive/MyDrive/chatbot_jardinagem/chatbot_jardinagem_novo2.db
🚀 WebDriver do Selenium inicializado.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5 [00:00<?, ? examples/s]

✅ Datasets de treino e teste pré-processados e salvos em 'dataset_jardinagem_flant5_pronto'

🚪 Conexão com o banco de dados fechada.

🎉 Script finalizado.


In [ ]:
# Célula para criar o ficheiro perguntas_manuais.csv no ambiente do Colab

conteudo_csv = """pergunta,resposta
"O que é um bonsai?","Um bonsai é uma árvore em miniatura cultivada num vaso ou recipiente raso. É a arte de cultivar árvores e arbustos de forma a que permaneçam pequenos, mas mantenham a aparência de árvores maduras na natureza. Isto é conseguido através de técnicas de poda contínua de raízes e ramos, aramação dos ramos para moldar a forma, e cuidados específicos com a rega e adubação. Um bonsai não é uma espécie de planta geneticamente anã; qualquer espécie de árvore com tronco lenhoso e ramos pode, teoricamente, ser transformada num bonsai."
"O que é um bonsai?","Um bonsai é uma árvore em miniatura cultivada num vaso ou recipiente raso. É a arte de cultivar árvores e arbustos de forma a que permaneçam pequenos, mas mantenham a aparência de árvores maduras na natureza. Isto é conseguido através de técnicas de poda contínua de raízes e ramos, aramação dos ramos para moldar a forma, e cuidados específicos com a rega e adubação. Um bonsai não é uma espécie de planta geneticamente anã; qualquer espécie de árvore com tronco lenhoso e ramos pode, teoricamente, ser transformada num bonsai."
"O que é um bonsai?","Um bonsai é uma árvore em miniatura cultivada num vaso ou recipiente raso. É a arte de cultivar árvores e arbustos de forma a que permaneçam pequenos, mas mantenham a aparência de árvores maduras na natureza. Isto é conseguido através de técnicas de poda contínua de raízes e ramos, aramação dos ramos para moldar a forma, e cuidados específicos com a rega e adubação. Um bonsai não é uma espécie de planta geneticamente anã; qualquer espécie de árvore com tronco lenhoso e ramos pode, teoricamente, ser transformada num bonsai."
"O que é um bonsai?","Um bonsai é uma árvore em miniatura cultivada num vaso ou recipiente raso. É a arte de cultivar árvores e arbustos de forma a que permaneçam pequenos, mas mantenham a aparência de árvores maduras na natureza. Isto é conseguido através de técnicas de poda contínua de raízes e ramos, aramação dos ramos para moldar a forma, e cuidados específicos com a rega e adubação. Um bonsai não é uma espécie de planta geneticamente anã; qualquer espécie de árvore com tronco lenhoso e ramos pode, teoricamente, ser transformada num bonsai."
"Como devo adubar tomates corretamente?","Para adubar tomates corretamente, comece com um solo rico em matéria orgânica. No plantio, pode usar um fertilizante balanceado ou um com maior teor de fósforo para estimular o desenvolvimento das raízes. Durante a fase de crescimento e frutificação, os tomates beneficiam de fertilizantes ricos em potássio, que ajuda na formação e qualidade dos frutos. Evite excesso de nitrogénio, que pode levar a muito crescimento de folhas em detrimento dos frutos. Uma opção é aplicar um fertilizante líquido a cada 2-4 semanas ou um granulado de libertação lenta conforme as instruções do fabricante."
"Como devo adubar tomates corretamente?","Para adubar tomates corretamente, comece com um solo rico em matéria orgânica. No plantio, pode usar um fertilizante balanceado ou um com maior teor de fósforo para estimular o desenvolvimento das raízes. Durante a fase de crescimento e frutificação, os tomates beneficiam de fertilizantes ricos em potássio, que ajuda na formação e qualidade dos frutos. Evite excesso de nitrogénio, que pode levar a muito crescimento de folhas em detrimento dos frutos. Uma opção é aplicar um fertilizante líquido a cada 2-4 semanas ou um granulado de libertação lenta conforme as instruções do fabricante."
"Como devo adubar tomates corretamente?","Para adubar tomates corretamente, comece com um solo rico em matéria orgânica. No plantio, pode usar um fertilizante balanceado ou um com maior teor de fósforo para estimular o desenvolvimento das raízes. Durante a fase de crescimento e frutificação, os tomates beneficiam de fertilizantes ricos em potássio, que ajuda na formação e qualidade dos frutos. Evite excesso de nitrogénio, que pode levar a muito crescimento de folhas em detrimento dos frutos. Uma opção é aplicar um fertilizante líquido a cada 2-4 semanas ou um granulado de libertação lenta conforme as instruções do fabricante."
"Como devo adubar tomates corretamente?","Para adubar tomates corretamente, comece com um solo rico em matéria orgânica. No plantio, pode usar um fertilizante balanceado ou um com maior teor de fósforo para estimular o desenvolvimento das raízes. Durante a fase de crescimento e frutificação, os tomates beneficiam de fertilizantes ricos em potássio, que ajuda na formação e qualidade dos frutos. Evite excesso de nitrogénio, que pode levar a muito crescimento de folhas em detrimento dos frutos. Uma opção é aplicar um fertilizante líquido a cada 2-4 semanas ou um granulado de libertação lenta conforme as instruções do fabricante."
"Qual a melhor época para podar hortênsias?","A melhor época para podar hortênsias depende do tipo. As hortênsias mais comuns (Hydrangea macrophylla), que florescem em madeira velha, devem ser podadas logo após a floração, geralmente no final do verão ou início do outono, removendo as flores murchas e, se necessário, alguns dos caules mais velhos para rejuvenescer a planta. Hortênsias que florescem em madeira nova (como Hydrangea paniculata e Hydrangea arborescens) podem ser podadas no final do inverno ou início da primavera, antes do início do novo crescimento."
"Como plantar alface?","Para plantar alface, escolha um local com pelo menos 4-6 horas de sol por dia e solo bem drenado e rico em matéria orgânica. As sementes podem ser semeadas diretamente no solo ou em sementeiras para transplante posterior. Cubra as sementes levemente com terra, pois precisam de luz para germinar. Mantenha o solo consistentemente húmido, mas não encharcado. O espaçamento varia conforme a variedade, mas geralmente é de 15-30 cm entre as plantas. A alface prefere climas mais amenos."
"O que são pulgões e como combatê-los?","Pulgões são pequenos insetos sugadores de seiva que podem infestar uma grande variedade de plantas, causando amarelecimento das folhas, deformações e transmissão de doenças. Para combatê-los, pode-se tentar remover manualmente com um jato de água, aplicar sabão inseticida ou óleo de neem, ou introduzir predadores naturais como joaninhas. Em infestações severas, inseticidas específicos podem ser necessários, mas sempre leia e siga as instruções do rótulo."
"O que são pulgões e como combatê-los?","Pulgões são pequenos insetos sugadores de seiva que podem infestar uma grande variedade de plantas, causando amarelecimento das folhas, deformações e transmissão de doenças. Para combatê-los, pode-se tentar remover manualmente com um jato de água, aplicar sabão inseticida ou óleo de neem, ou introduzir predadores naturais como joaninhas. Em infestações severas, inseticidas específicos podem ser necessários, mas sempre leia e siga as instruções do rótulo."
"O que são pulgões e como combatê-los?","Pulgões são pequenos insetos sugadores de seiva que podem infestar uma grande variedade de plantas, causando amarelecimento das folhas, deformações e transmissão de doenças. Para combatê-los, pode-se tentar remover manualmente com um jato de água, aplicar sabão inseticida ou óleo de neem, ou introduzir predadores naturais como joaninhas. Em infestações severas, inseticidas específicos podem ser necessários, mas sempre leia e siga as instruções do rótulo."
"O que são pulgões e como combatê-los?","Pulgões são pequenos insetos sugadores de seiva que podem infestar uma grande variedade de plantas, causando amarelecimento das folhas, deformações e transmissão de doenças. Para combatê-los, pode-se tentar remover manualmente com um jato de água, aplicar sabão inseticida ou óleo de neem, ou introduzir predadores naturais como joaninhas. Em infestações severas, inseticidas específicos podem ser necessários, mas sempre leia e siga as instruções do rótulo."
"""

# Nome do ficheiro a ser criado no Colab
nome_ficheiro_csv = "perguntas_manuais.csv"

with open(nome_ficheiro_csv, "w", encoding="utf-8") as f:
    f.write(conteudo_csv)

print(f"Ficheiro '{nome_ficheiro_csv}' criado com sucesso no diretório atual do Colab (/content/).")

Ficheiro 'perguntas_manuais.csv' criado com sucesso no diretório atual do Colab (/content/).


Ajustes no pré-processamento e no F-T

In [ ]:
import os
import sqlite3
import re
import random
import csv # NOVO: Para ler o ficheiro CSV
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer
import argparse
import yake

# --- Funções Auxiliares ---

# Função de Limpeza (Mantida como antes - sem alterações)
def limpar_conteudo_treino(conteudo):
    # ... (código original da função)
    if not conteudo: return ""
    conteudo = re.sub(r"https?://\S+|www\.[a-zA-Z0-9\-\.]+\.[a-zA-Z]+", "", conteudo)
    boilerplate_patterns = [
        r"usamos cookies.*?(?:\.|$)", r"compartilhe.*?(?:\.|$)",
        r"política de privacidade.*?(?:\.|$)", r"leia mais.*?(?:\.|$)",
        r"veja também.*?(?:\.|$)", r"fontes?:.*?(?:\.|$)",
        r"imagens?:.*?(?:\.|$)", r"créditos?:.*?(?:\.|$)",
        r"^\s*publicidade\s*$", r"assine nossa newsletter.*?(?:\.|$)",
        r"^\s*Vasyl Cherlinka é Doutor.*?Universidade Nacional de Chernivtsi\.",
        r"^\s*Cadastre-se hoje.*?últimos 3 meses\.",
        r"^\s*As laranjas têm alta demanda.*?tornando o cultivo lucrativo\.",
        r"^\s*Os ácaros aranha.*?agrícolas comerciais\.",
        r"^\s*A mandioca cresce bem.*?grande investimento\.",
        r"^\s*Comentários\s*$", r"^\s*Deixe um comentário\s*[:.]?$",
        r"^\s*Nome\s*$", r"^\s*Email\s*$", r"^\s*Website\s*$",
        r"Salvar meus dados neste navegador.*?(?:\.|$)",
    ]
    for pattern in boilerplate_patterns:
        conteudo = re.sub(pattern, "", conteudo, flags=re.IGNORECASE | re.MULTILINE)
    conteudo = re.sub(r'^\s*\n', '', conteudo, flags=re.MULTILINE)
    conteudo = re.sub(r"[^\w,.!?-çáéíóúãõâêîôûàèìòùüÁÉÍÓÚÃÕÂÊÎÔÛÀÈÌÒÙÜ%\n\s]", "", conteudo, flags=re.UNICODE)
    conteudo = re.sub(r"[ \t]+", " ", conteudo)
    conteudo = re.sub(r"^[ \t]+|[ \t]+$", "", conteudo, flags=re.MULTILINE)
    conteudo = re.sub(r"\n{3,}", "\n\n", conteudo)
    conteudo = conteudo.strip()
    return conteudo

# Função para dividir texto em Chunks (Mantida como antes - sem alterações)
def dividir_em_chunks(texto, separador="\n", min_chars_por_chunk=100):
    # ... (código original da função)
    if not texto:
        return []
    chunks_inicial = texto.split(separador)
    chunks_final = []
    for chunk in chunks_inicial:
        chunk_limpo = chunk.strip()
        if chunk_limpo and len(chunk_limpo) >= min_chars_por_chunk:
            chunks_final.append(chunk_limpo)
    return chunks_final

# --- ALTERAÇÃO SIGNIFICATIVA: Função para Gerar Perguntas com Mais Variedade ---
kw_extrator_chunk = yake.KeywordExtractor(lan="pt", n=3, dedupLim=0.9, top=3, features=None) # Ajustado para mais palavras-chave

def gerar_pergunta_QeA_para_chunk(titulo_original, chunk_texto):
    """
    Gera uma pergunta no estilo Q&A, onde o chunk_texto serve como resposta.
    Agora com mais variedade de templates.
    """
    titulo_limpo = titulo_original.strip().lower()
    topico_principal = re.sub(r"^(como plantar|guia completo|dicas para|tudo sobre|o que é|saiba como|descubra|entenda|aprenda a)\s+", "", titulo_limpo, flags=re.IGNORECASE)
    topico_principal = re.split(r"\s*[:\-–—|(]\s*", topico_principal)[0].strip()
    topico_principal = re.sub(r"\s*-\s*jardim das ideias.*|\s*–\s*jardim das ideias.*|\s*-\s*jardineiro\.net.*|wikihow.*", "", topico_principal, flags=re.IGNORECASE).strip()
    if topico_principal:
        topico_principal = topico_principal[0].upper() + topico_principal[1:]

    if not chunk_texto:
        return f"Fale mais sobre {topico_principal if topico_principal else 'jardinagem em geral'}."

    keywords_data = kw_extrator_chunk.extract_keywords(chunk_texto)
    palavras_chave_chunk = [kw for kw, score in keywords_data[:3]] # Pega até 3 palavras-chave

    if not topico_principal and palavras_chave_chunk:
        topico_principal = random.choice(palavras_chave_chunk) # Usa uma palavra-chave aleatória como tópico
        topico_principal = topico_principal[0].upper() + topico_principal[1:]
    elif not topico_principal:
        topico_principal = "jardinagem"

    # Templates gerais e de definição
    templates_definicao_geral = [
        "O que é {topico_principal}?",
        "Pode me explicar o que significa {topico_principal}?",
        "Defina {topico_principal}.",
        "Fale mais sobre {topico_principal}.",
        "Quais são os pontos principais sobre {topico_principal}?",
        "Descreva {topico_principal} em detalhes.",
        "Gostaria de saber mais sobre {topico_principal}.",
        "Explique {topico_principal}."
    ]

    # Templates baseados em palavras-chave (pk1)
    templates_pk1 = [
        "Como devo fazer para {palavra_chave_1} em {topico_principal}?",
        "Quais são as dicas sobre {palavra_chave_1} ao cuidar de {topico_principal}?",
        "O que é importante saber sobre {palavra_chave_1} para {topico_principal}?",
        "Explique como funciona {palavra_chave_1} no contexto de {topico_principal}.",
        "Qual a melhor forma de lidar com {palavra_chave_1} em {topico_principal}?",
        "Me fale sobre {palavra_chave_1} quando se trata de {topico_principal}.",
        "Pode me explicar {palavra_chave_1} relacionado a {topico_principal}?",
        "Quais os passos para {palavra_chave_1} em {topico_principal}?",
        "Qual a importância de {palavra_chave_1} para {topico_principal}?"
    ]

    # Templates baseados em duas palavras-chave (pk1, pk2)
    templates_pk2 = [
        "Qual a relação entre {palavra_chave_1} e {palavra_chave_2} em {topico_principal}?",
        "Como {palavra_chave_1} influencia {palavra_chave_2} ao tratar de {topico_principal}?",
        "Ao cuidar de {topico_principal}, como devo considerar {palavra_chave_1} e {palavra_chave_2} juntos?",
        "Quais informações você tem sobre {palavra_chave_1} e {palavra_chave_2} para {topico_principal}?"
    ]

    pergunta_final = ""
    chance = random.random() # Número entre 0.0 e 1.0

    if not palavras_chave_chunk or chance < 0.25: # 25% de chance de usar template de definição/geral
        pergunta_final = random.choice(templates_definicao_geral).format(topico_principal=topico_principal)
    elif len(palavras_chave_chunk) == 1 or chance < 0.65: # +40% = 65% de chance de usar template com pk1
        pk1 = palavras_chave_chunk[0]
        pergunta_final = random.choice(templates_pk1).format(palavra_chave_1=pk1, topico_principal=topico_principal)
    else: # Restante (35%) de chance de usar template com pk2 (se pk2 existir)
        pk1 = palavras_chave_chunk[0]
        if len(palavras_chave_chunk) >= 2:
            pk2 = palavras_chave_chunk[1]
            template_escolhido = random.choice(templates_pk2)
            pergunta_final = template_escolhido.format(palavra_chave_1=pk1, palavra_chave_2=pk2, topico_principal=topico_principal)
        else: # Fallback se não tiver pk2
            pergunta_final = random.choice(templates_pk1).format(palavra_chave_1=pk1, topico_principal=topico_principal)

    pergunta_final = pergunta_final.strip()
    if pergunta_final and not pergunta_final.endswith("?"):
        pergunta_final += "?"

    return pergunta_final.capitalize()


# --- NOVA FUNÇÃO: Carregar perguntas e respostas manuais de um CSV ---
def carregar_dados_manuais_csv(csv_path):
    """Carrega pares de pergunta/resposta de um ficheiro CSV."""
    dataset_lista_manual = []
    if not os.path.exists(csv_path):
        print(f"ℹ️  Aviso: Ficheiro de dados manuais '{csv_path}' não encontrado. Apenas dados do banco serão usados.")
        return dataset_lista_manual

    try:
        with open(csv_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            if "pergunta" not in reader.fieldnames or "resposta" not in reader.fieldnames:
                print(f"❌ Erro: O ficheiro CSV '{csv_path}' deve conter as colunas 'pergunta' e 'resposta'.")
                return dataset_lista_manual

            for i, row in enumerate(reader):
                pergunta = row.get("pergunta", "").strip()
                resposta = row.get("resposta", "").strip()
                if pergunta and resposta:
                    dataset_lista_manual.append({
                        "id": f"manual_{i+1}",
                        "pergunta": pergunta,
                        "resposta": resposta
                    })
            print(f"📄 Carregados {len(dataset_lista_manual)} pares de pergunta/resposta do ficheiro '{csv_path}'.")
    except Exception as e:
        print(f"❌ Erro ao ler o ficheiro CSV '{csv_path}': {e}")
    return dataset_lista_manual

# --- Função de Carregamento do Banco (AJUSTADA para integrar dados manuais) ---
def carregar_dados_brutos_banco_e_manuais(db_path, csv_manual_path, min_chunk_len=150):
    """
    Carrega dados do banco, limpa, divide em chunks, gera pares pergunta/resposta sintéticos
    E ADICIONA dados de um ficheiro CSV manual.
    """
    dataset_lista = []

    # 1. Carregar dados manuais primeiro
    dados_manuais = carregar_dados_manuais_csv(csv_manual_path)
    dataset_lista.extend(dados_manuais)

    # 2. Carregar e processar dados do banco
    conn = None
    registros_processados_db = 0
    pares_gerados_db = 0
    try:
        if not os.path.exists(db_path):
            print(f"❌ Erro: Arquivo do banco de dados não encontrado em {db_path}")
            if not dataset_lista: # Se não houver dados manuais também, retorna lista vazia.
                 return []
        else: # Só processa o banco se ele existir
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()
            cursor.execute("SELECT titulo, conteudo_completo FROM artigos WHERE conteudo_completo IS NOT NULL AND LENGTH(TRIM(conteudo_completo)) > 0")
            dados_db = cursor.fetchall()
            print(f"📄 Carregados {len(dados_db)} registros do banco para processamento sintético.")

            for i, (titulo, conteudo_completo) in enumerate(dados_db):
                if not titulo or not conteudo_completo:
                    continue

                print(f"\n[{i+1}/{len(dados_db)}] Processando (DB): {titulo[:70]}...")
                conteudo_limpo = limpar_conteudo_treino(conteudo_completo)
                if not conteudo_limpo:
                    print("     ⚠️ Conteúdo do DB ficou vazio após limpeza.")
                    continue

                chunks = dividir_em_chunks(conteudo_limpo, min_chars_por_chunk=min_chunk_len)
                if not chunks:
                    print(f"     ⚠️ Nenhum chunk encontrado no DB com tamanho >= {min_chunk_len} chars.")
                    continue

                registros_processados_db += 1
                for j, chunk in enumerate(chunks):
                    pergunta = gerar_pergunta_QeA_para_chunk(titulo, chunk) # Usa a função melhorada
                    resposta_chunk = chunk

                    dataset_lista.append({
                        "id": f"db_art{i+1}_chunk{j+1}",
                        "pergunta": pergunta,
                        "resposta": resposta_chunk
                    })
                    pares_gerados_db += 1
                    if j < 1 : # Imprimir o primeiro exemplo gerado por artigo
                        print(f"       Exemplo Pergunta (DB Sintético): {pergunta}")
                        # print(f"       Exemplo Resposta (Chunk DB): {resposta_chunk[:150]}...")

            print(f"\n📊 Processados {registros_processados_db} artigos do banco.")
            print(f"📊 Gerados {pares_gerados_db} pares pergunta/chunk sintéticos do banco.")

    except sqlite3.Error as e:
        print(f"❌ Erro ao acessar o banco de dados: {e}")
    finally:
        if conn:
            conn.close()

    print(f"📊 Total de {len(dataset_lista)} pares pergunta/resposta (manuais + sintéticos) prontos para tokenização.")
    return dataset_lista


# --- Função de Pré-processamento T5 para Q&A (Mantida como antes - sem alterações) ---
def preprocessar_dados_t5_QeA(exemplos, tokenizer, max_input_length=512, max_target_length=256):
    # ... (código original da função)
    prefixo_qa = "pergunta: "
    inputs = [prefixo_qa + str(p) for p in exemplos["pergunta"]]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        padding="max_length",
        truncation=True
    )
    labels = tokenizer(
        text_target=exemplos["resposta"],
        max_length=max_target_length,
        padding="max_length",
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# --- Lógica Principal (AJUSTADA para chamar a nova função de carregamento e incluir path do CSV) ---
def main(db_path, csv_manual_path, output_path, model_name, test_size=0.1, seed=42, max_input=512, max_target=256, min_chunk=150):
    print(f"--- Iniciando Preparação do Dataset Q&A (min chunk sintético: {min_chunk} chars, max target (resposta) len: {max_target} tokens) ---")

    # ALTERADO: Chamar a nova função que carrega dados do banco E do CSV manual
    dados_brutos_completos = carregar_dados_brutos_banco_e_manuais(db_path, csv_manual_path, min_chunk_len=min_chunk)

    if not dados_brutos_completos:
        print("🛑 Nenhum dado (manual ou do banco) encontrado ou gerado para Q&A. Encerrando.")
        return

    dataset_hf = Dataset.from_list(dados_brutos_completos)
    # Embaralhar o dataset antes de dividir, para misturar dados manuais e sintéticos
    dataset_hf = dataset_hf.shuffle(seed=seed)
    print(f"📊 Dataset Hugging Face TOTAL para Q&A criado com {len(dataset_hf)} exemplos (manuais + sintéticos).")

    if len(dataset_hf) > 10:
        dataset_splits = dataset_hf.train_test_split(test_size=test_size, seed=seed) # Seed aqui garante a mesma divisão
        train_dataset = dataset_splits["train"]
        eval_dataset = dataset_splits["test"]
        print(f"   Divisão em {len(train_dataset)} amostras de treino e {len(eval_dataset)} de teste.")
    else:
        print("⚠️ Dataset muito pequeno, usando tudo para treino e criando teste vazio.")
        train_dataset = dataset_hf
        eval_dataset = Dataset.from_dict(train_dataset.features.copy()).select([]) if train_dataset.features else Dataset.from_list([])

    print(f"🔄 Carregando tokenizer para {model_name}...")
    try:
        tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
    except Exception as e:
        print(f"❌ Falha ao carregar tokenizer {model_name}: {e}")
        return

    print("⚙️ Aplicando pré-processamento T5 (Q&A) aos dados...")
    remove_cols = ["pergunta", "resposta", "id"]

    train_dataset_processed = train_dataset.map(
        preprocessar_dados_t5_QeA,
        batched=True,
        fn_kwargs={"tokenizer": tokenizer, "max_input_length": max_input, "max_target_length": max_target},
        remove_columns=[col for col in remove_cols if col in train_dataset.column_names]
    )
    eval_dataset_processed = eval_dataset.map(
        preprocessar_dados_t5_QeA,
        batched=True,
        fn_kwargs={"tokenizer": tokenizer, "max_input_length": max_input, "max_target_length": max_target},
        remove_columns=[col for col in remove_cols if col in eval_dataset.column_names]
    )

    processed_dataset_dict = DatasetDict({
        "train": train_dataset_processed,
        "test": eval_dataset_processed
    })

    print(f"💾 Salvando dataset Q&A processado em '{output_path}'...")
    try:
        os.makedirs(output_path, exist_ok=True)
        processed_dataset_dict.save_to_disk(output_path)
        print(f"✅ Dataset Q&A processado e salvo com sucesso em '{output_path}'.")
        print("   Este dataset está pronto para ser usado no script de fine-tuning para um modelo Q&A.")
    except Exception as e:
        print(f"❌ Erro ao salvar o dataset Q&A processado: {e}")

# --- Execução Principal (AJUSTADA para incluir argumento do CSV manual) ---
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Prepara dataset Q&A para fine-tuning T5 a partir de SQLite e CSV manual.")
    parser.add_argument("--db_path", type=str, default="chatbot_jardinagem_novo2.db", help="Caminho para o banco SQLite.")

    # NOVO ARGUMENTO: Caminho para o ficheiro CSV de perguntas manuais
    parser.add_argument("--csv_manual_path", type=str, default="perguntas_manuais.csv", help="Caminho para o ficheiro CSV com perguntas/respostas manuais.")

    parser.add_argument("--output_path", type=str, default="./processed_dataset_jardinagem_QeA", help="Pasta de saída para o dataset Q&A processado.")
    parser.add_argument("--model_name", type=str, default="google/flan-t5-base", help="Nome do modelo T5 para usar o tokenizer.")
    parser.add_argument("--test_size", type=float, default=0.1, help="Proporção para teste/validação.")
    parser.add_argument("--seed", type=int, default=42, help="Seed para divisão treino/teste e shuffle.")
    parser.add_argument("--max_input", type=int, default=512, help="Max length para input tokenizado.")
    parser.add_argument("--max_target", type=int, default=256, help="Max length para target (resposta) tokenizado.")
    parser.add_argument("--min_chunk", type=int, default=150, help="Tamanho mínimo (em caracteres) para um chunk sintético ser considerado válido.")

    args, unknown_args = parser.parse_known_args()
    if unknown_args:
        print(f"⚠️ Argumentos desconhecidos ignorados: {unknown_args}")

    db_full_path = args.db_path
    # Tenta encontrar o DB no Google Drive
    if not os.path.exists(db_full_path) and os.path.exists("/content/drive/MyDrive"):
        drive_path_base = "/content/drive/MyDrive/chatbot_jardinagem" # Ajuste se necessário
        drive_db_path_try = os.path.join(drive_path_base, os.path.basename(args.db_path))
        if os.path.exists(drive_db_path_try):
            print(f"Ajustando caminho do DB para Google Drive: {drive_db_path_try}")
            db_full_path = drive_db_path_try
        # else: # Não emite aviso aqui, a função carregar_dados_brutos_banco_e_manuais fará isso

    csv_manual_full_path = args.csv_manual_path
    # Tenta encontrar o CSV manual no Google Drive
    if not os.path.exists(csv_manual_full_path) and os.path.exists("/content/drive/MyDrive"):
        drive_path_base_csv = os.path.dirname(db_full_path) # Tenta a mesma pasta do DB no Drive
        drive_csv_path_try = os.path.join(drive_path_base_csv, os.path.basename(args.csv_manual_path))
        if os.path.exists(drive_csv_path_try):
            print(f"Ajustando caminho do CSV manual para Google Drive: {drive_csv_path_try}")
            csv_manual_full_path = drive_csv_path_try
        # else: # Não emite aviso aqui, a função carregar_dados_manuais_csv fará isso

    main(db_full_path, csv_manual_full_path, args.output_path, args.model_name, args.test_size, args.seed, args.max_input, args.max_target, args.min_chunk)

⚠️ Argumentos desconhecidos ignorados: ['-f', '/root/.local/share/jupyter/runtime/kernel-0f510d46-54a9-4b18-b326-ff915f99065e.json']
Ajustando caminho do DB para Google Drive: /content/drive/MyDrive/chatbot_jardinagem/chatbot_jardinagem_novo2.db
--- Iniciando Preparação do Dataset Q&A (min chunk sintético: 150 chars, max target (resposta) len: 256 tokens) ---
📄 Carregados 14 pares de pergunta/resposta do ficheiro 'perguntas_manuais.csv'.
📄 Carregados 22 registros do banco para processamento sintético.

[1/22] Processando (DB): INTRODUÇÃO AO BONSAI...
       Exemplo Pergunta (DB Sintético): Quais são os pontos principais sobre introdução ao bonsai?

[2/22] Processando (DB): Dicas práticas para transformar o seu jardim – Jardim das Ideias STIHL...
       Exemplo Pergunta (DB Sintético): Qual a relação entre jardim e livre em dicas práticas para transformar o seu jardim?

[3/22] Processando (DB): ÁCAROS...
       Exemplo Pergunta (DB Sintético): Explique como funciona cor laranja no conte

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

💾 Salvando dataset Q&A processado em './processed_dataset_jardinagem_QeA'...


Saving the dataset (0/1 shards):   0%|          | 0/457 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/51 [00:00<?, ? examples/s]

✅ Dataset Q&A processado e salvo com sucesso em './processed_dataset_jardinagem_QeA'.
   Este dataset está pronto para ser usado no script de fine-tuning para um modelo Q&A.


Visualização de perguntas e respostas gerada spelo pré-processamento

Ajustes no fine-tuning

In [ ]:
import os
import torch
import nltk
import numpy as np
import evaluate # Use a biblioteca evaluate
from datasets import load_from_disk, DatasetDict
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    GenerationConfig # Importar GenerationConfig
)
import argparse
import logging
import transformers # Para verificar a versão
import inspect # Para verificar assinatura

# Configurar logging básico
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Download NLTK Punkt resources (Mantido) ---
punkt_resource = "tokenizers/punkt"
punkt_tab_resource = "tokenizers/punkt_tab"
try:
    nltk.data.find(punkt_resource)
    logging.info(f"Recurso NLTK '{os.path.basename(punkt_resource)}' já está disponível.")
except LookupError:
    logging.info(f"Baixando recurso '{os.path.basename(punkt_resource)}' do NLTK...")
    nltk.download("punkt", quiet=True)
    logging.info(f"Download do '{os.path.basename(punkt_resource)}' concluído.")
try:
    nltk.data.find(punkt_tab_resource)
    logging.info(f"Recurso NLTK '{os.path.basename(punkt_tab_resource)}' já está disponível.")
except LookupError:
    logging.info(f"Baixando recurso '{os.path.basename(punkt_tab_resource)}' do NLTK (necessário para sent_tokenize)...")
    nltk.download("punkt_tab", quiet=True)
    logging.info(f"Download do '{os.path.basename(punkt_tab_resource)}' concluído.")

# --- Lógica Principal do Fine-tuning ---
def main(processed_dataset_path, model_name, output_model_dir, logging_dir,
         epochs, batch_size, grad_accum, lr, fp16_enabled,
         generation_max_length_setting_default=256):

    logging.info(f"--- Iniciando Fine-tuning para Modelo Q&A ---")
    # ... (logs iniciais mantidos) ...

    try:
        logging.info("🔄 Carregando métrica ROUGE...")
        rouge_metric = evaluate.load("rouge")
        logging.info("✅ Métrica ROUGE carregada.")
    except Exception as e:
        # ... (tratamento de erro mantido) ...
        return

    # ... (carregamento do dataset mantido) ...
    logging.info(f"🔄 Carregando dataset pré-processado de '{processed_dataset_path}'...")
    if not os.path.exists(processed_dataset_path):
        # ... (tratamento de erro mantido) ...
        return
    try:
        tokenized_datasets = load_from_disk(processed_dataset_path)
        # ... (validação do dataset mantida) ...
        train_dataset = tokenized_datasets["train"]
        eval_dataset = tokenized_datasets.get("test")
        # ... (logs de dataset mantidos) ...
    except Exception as e:
        logging.error(f"❌ Erro ao carregar dataset de '{processed_dataset_path}': {e}")
        return

    logging.info(f"🔄 Carregando modelo e tokenizer base: {model_name}...")
    try:
        tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
        model = T5ForConditionalGeneration.from_pretrained(model_name)
        logging.info("✅ Modelo e tokenizer carregados.")

        # Definir os valores desejados para geração
        generation_max_length_to_set = generation_max_length_setting_default
        generation_num_beams_to_set = 4 # Você pode tornar isso um argumento se quiser

        # Assegurar que model.generation_config exista e esteja configurado
        if not hasattr(model, "generation_config") or model.generation_config is None:
            logging.warning("⚠️ model.generation_config não encontrado ou é None. Criando um default.")
            model.generation_config = GenerationConfig.from_model_config(model.config)

        model.generation_config.max_length = generation_max_length_to_set
        model.generation_config.num_beams = generation_num_beams_to_set

        log_msg = (f"✅ Parâmetros de geração definidos em model.generation_config: "
                   f"max_length={model.generation_config.max_length}, "
                   f"num_beams={model.generation_config.num_beams}")
        logging.info(log_msg)

    except Exception as e:
        logging.error(f"❌ Falha ao carregar modelo/tokenizer {model_name}: {e}")
        return

    # ... (compute_metrics e data_collator mantidos) ...
    def compute_metrics(eval_preds):
        # ... (código mantido) ...
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
        decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
        result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        result = {key: value * 100 if isinstance(value, float) else value for key, value in result.items()}
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        return {k: round(v, 4) for k, v in result.items()}

    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        label_pad_token_id=-100,
        pad_to_multiple_of=8 if fp16_enabled and torch.cuda.is_available() else None
    )

    fp16_actual = fp16_enabled and torch.cuda.is_available()
    # ... (logs de fp16 mantidos) ...

    do_eval = eval_dataset is not None
    metric_to_optimize = "rougeL" if do_eval else None
    load_best_val = do_eval
    greater_is_better_val = True if do_eval and metric_to_optimize != "eval_loss" else False

    training_args = TrainingArguments(
        output_dir=output_model_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        gradient_accumulation_steps=grad_accum,
        learning_rate=lr,
        fp16=fp16_actual,
        weight_decay=0.01,
        logging_dir=logging_dir,
        report_to="tensorboard",
        logging_strategy="epoch",
        eval_strategy="epoch" if do_eval else "no",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=load_best_val,
        metric_for_best_model=metric_to_optimize,
        greater_is_better=greater_is_better_val
    )

    # --- WORKAROUND PARA ATRIBUTOS DE GERAÇÃO EM TrainingArguments ---
    # Necessário para versões mais antigas do Transformers onde Seq2SeqTrainer
    # espera esses atributos diretamente em args, mesmo que model.generation_config esteja definido.
    logging.info("🐒 Aplicando workaround para atributos de geração em TrainingArguments...")

    # 1. Garantir training_args.generation_config
    if not hasattr(training_args, 'generation_config'):
        if hasattr(model, 'generation_config') and model.generation_config is not None:
            setattr(training_args, 'generation_config', model.generation_config)
            logging.info("   -> 'generation_config' copiado de model.generation_config para training_args.")
        else:
            # Criar um GenerationConfig default se o modelo não tiver um (improvável, mas seguro)
            # ou se model.generation_config for None.
            gen_conf_fallback = GenerationConfig.from_model_config(model.config) if hasattr(model, 'config') else GenerationConfig()
            gen_conf_fallback.max_length = generation_max_length_to_set # Aplicar nossos defaults
            gen_conf_fallback.num_beams = generation_num_beams_to_set   # Aplicar nossos defaults
            setattr(training_args, 'generation_config', gen_conf_fallback)
            logging.info(f"   -> 'generation_config' definido em training_args com defaults (max_length={generation_max_length_to_set}, num_beams={generation_num_beams_to_set}).")

    # 2. Garantir training_args.generation_max_length
    if not hasattr(training_args, 'generation_max_length') or getattr(training_args, 'generation_max_length', None) is None:
        setattr(training_args, 'generation_max_length', generation_max_length_to_set)
        logging.info(f"   -> 'generation_max_length' definido/atualizado para {generation_max_length_to_set} em training_args.")

    # 3. Garantir training_args.generation_num_beams
    if not hasattr(training_args, 'generation_num_beams') or getattr(training_args, 'generation_num_beams', None) is None:
        setattr(training_args, 'generation_num_beams', generation_num_beams_to_set)
        logging.info(f"   -> 'generation_num_beams' definido/atualizado para {generation_num_beams_to_set} em training_args.")

    # 4. Garantir predict_with_generate (para versões muito antigas que podem não ter isso como True por padrão no Seq2SeqTrainer)
    if not hasattr(training_args, 'predict_with_generate'):
        try:
            setattr(training_args, 'predict_with_generate', True)
            logging.info("   -> Atributo 'predict_with_generate' definido manualmente como True em training_args.")
        except Exception as e_setattr_pwg:
            logging.warning(f"   -> Não foi possível definir 'predict_with_generate' manualmente em training_args: {e_setattr_pwg}")
    # --- FIM DO WORKAROUND ---

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics if do_eval else None,
        tokenizer=tokenizer
    )

    # ... (lógica de treinamento e avaliação mantida) ...
    logging.info(f"🚀 Iniciando treinamento para {epochs} épocas...")
    try:
        train_result = trainer.train()
        # ... (resto do código de salvar e avaliar mantido) ...
        logging.info("✅ Treinamento concluído.")
        trainer.log_metrics("train", train_result.metrics)
        trainer.save_metrics("train", train_result.metrics)
        trainer.save_state()
        logging.info(f"💾 Salvando modelo final (melhor checkpoint) em '{output_model_dir}'...")
        trainer.save_model(output_model_dir)
        logging.info("✅ Modelo e estado do trainer salvos.")

        if do_eval:
            logging.info("📊 Realizando avaliação final no dataset de teste com o melhor modelo...")
            eval_metrics = trainer.evaluate()
            if "eval_gen_len" in eval_metrics:
                eval_metrics["eval_gen_len"] = round(eval_metrics["eval_gen_len"], 4)
            logging.info(f"   Métricas de Avaliação Final: {eval_metrics}")
            trainer.log_metrics("eval", eval_metrics)
            trainer.save_metrics("eval", eval_metrics)
            logging.info("✅ Avaliação final concluída.")
    except Exception as e:
        logging.error(f"❌ Erro durante o treinamento/avaliação: {e}", exc_info=True)


# ---- Bloco Principal (if __name__ == "__main__") ----
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Script de Fine-tuning para Modelos T5 Seq2Seq para Q&A")
    # ... (argumentos mantidos como na última versão) ...
    parser.add_argument("--processed_dataset_path", type=str,
                        default="./processed_dataset_jardinagem_QeA",
                        help="Caminho para o dataset Q&A processado.")
    parser.add_argument("--model_name", type=str,
                        default="google/flan-t5-base",
                        help="Nome ou caminho do modelo T5 base.")
    parser.add_argument("--output_model_dir", type=str,
                        default="/content/drive/MyDrive/chatbot_jardinagem/modelo_QeA_ajustado",
                        help="Diretório para salvar o modelo Q&A ajustado (RECOMENDADO: Google Drive).")
    parser.add_argument("--logging_dir", type=str,
                        default="./logs_jardinagem_QeA",
                        help="Diretório para salvar logs do TensorBoard para Q&A.")
    parser.add_argument("--epochs", type=int,
                        default=8,
                        help="Número de épocas de treinamento (recomendado 5-10 para Q&A).")
    parser.add_argument("--batch_size", type=int, default=4,
                        help="Tamanho do lote por dispositivo (GPU/CPU). Ajuste conforme a memória da GPU.")
    parser.add_argument("--grad_accum", type=int, default=2,
                        help="Passos de acumulação de gradiente.")
    parser.add_argument("--lr", type=float, default=3e-5,
                        help="Taxa de aprendizado.")
    parser.add_argument("--fp16", action='store_true',
                        help="Habilitar treinamento com precisão mista (FP16) se CUDA disponível.")
    parser.add_argument("--generation_max_length", type=int,
                        default=256,
                        help="Comprimento máximo para geração de respostas (definido no model.generation_config).")

    args = parser.parse_args([] if "__file__" not in locals() else None)

    # ... (lógica de criação de diretórios mantida) ...
    if args.output_model_dir.startswith('/content/drive'):
        if not os.path.exists(os.path.dirname(args.output_model_dir)):
            logging.warning(f"Diretório pai '{os.path.dirname(args.output_model_dir)}' não parece existir. Tentando criar...")
            try:
                os.makedirs(os.path.dirname(args.output_model_dir), exist_ok=True)
                logging.info(f"Diretório pai '{os.path.dirname(args.output_model_dir)}' criado.")
            except Exception as e:
                logging.error(f"Falha ao criar diretório pai '{os.path.dirname(args.output_model_dir)}': {e}")
                logging.warning("Certifique-se que o Google Drive está montado e o caminho base existe.")
        if not os.path.exists(args.output_model_dir):
             os.makedirs(args.output_model_dir, exist_ok=True)
             logging.info(f"Diretório de saída do modelo '{args.output_model_dir}' criado.")
    elif not os.path.exists(args.output_model_dir):
        logging.info(f"Diretório de saída '{args.output_model_dir}' não está no Google Drive e não existe. Criando localmente...")
        os.makedirs(args.output_model_dir, exist_ok=True)
    if not os.path.exists(args.logging_dir):
        os.makedirs(args.logging_dir, exist_ok=True)


    main(
        args.processed_dataset_path,
        args.model_name,
        args.output_model_dir,
        args.logging_dir,
        args.epochs,
        args.batch_size,
        args.grad_accum,
        args.lr,
        args.fp16,
        args.generation_max_length
    )

    logging.info("\n🎉 Processo de Fine-tuning Q&A Finalizado!")

<ipython-input-48-9e2f148c89ad>:189: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,9.494900,3.354226,11.302300,2.143000,9.784800,10.028100,243.764700
2,3.327600,2.601733,10.600600,1.940600,9.243600,9.755900,255.000000
3,2.563000,2.073201,10.437700,1.949300,9.415800,9.588600,255.000000
4,2.081600,1.972415,11.521100,2.397200,10.389100,10.184600,255.000000
5,1.926400,1.927695,11.846400,2.645300,11.040700,10.809700,255.000000
6,1.869400,1.903346,11.996400,2.707600,10.813200,10.792600,255.000000
7,1.843100,1.887931,12.645500,2.678400,11.203900,11.379700,255.000000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


***** train metrics *****
  epoch                    =     7.8696
  total_flos               =  2295190GF
  train_loss               =     3.1398
  train_runtime            = 0:11:16.97
  train_samples_per_second =        5.4
  train_steps_per_second   =      0.674


***** eval metrics *****
  epoch                   =     7.8696
  eval_gen_len            =      255.0
  eval_loss               =     1.8879
  eval_rouge1             =    12.6455
  eval_rouge2             =     2.6784
  eval_rougeL             =    11.2039
  eval_rougeLsum          =    11.3797
  eval_runtime            = 0:00:53.02
  eval_samples_per_second =      0.962
  eval_steps_per_second   =      0.132


In [ ]:
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import gradio as gr
import logging

# Configurar logging básico
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configurações do Modelo ---
modelo_path = "/content/drive/MyDrive/chatbot_jardinagem/modelo_QeA_ajustado"


# --- Variáveis Globais para Modelo e Tokenizer ---
tokenizer_global = None
model_global = None
modelo_carregado = False

# --- Função para Carregar o Modelo e Tokenizer ---
def carregar_modelo_e_tokenizer():
    global tokenizer_global, model_global, modelo_carregado
    if modelo_carregado:
        logging.info("Modelo Q&A e tokenizer já estavam carregados.")
        return

    logging.info(f"🔄 Tentando carregar tokenizer e modelo Q&A de '{modelo_path}'...")
    if not os.path.isdir(modelo_path):
        logging.error(f"❌ ERRO CRÍTICO: A pasta do modelo Q&A '{modelo_path}' não foi encontrada.")
        logging.error("   Verifique o caminho e se o fine-tuning do modelo Q&A foi concluído e salvo corretamente.")
        raise FileNotFoundError(f"Diretório do modelo Q&A não encontrado: {modelo_path}.")

    try:
        tokenizer_global = T5Tokenizer.from_pretrained(modelo_path, legacy=False)
        model_global = T5ForConditionalGeneration.from_pretrained(modelo_path)
        modelo_carregado = True
        logging.info("✅ Tokenizer e modelo Q&A carregados com sucesso globalmente.")
    except Exception as e:
        logging.error(f"❌ ERRO CRÍTICO ao carregar modelo/tokenizer Q&A de '{modelo_path}': {e}")
        raise RuntimeError(f"Falha ao carregar modelo/tokenizer Q&A: {e}. Verifique os arquivos do modelo.")

# --- Função para Gerar a Resposta ---
def gerar_resposta_chatbot(pergunta_usuario):
    global tokenizer_global, model_global, modelo_carregado #, device

    if not modelo_carregado:
        try:
            carregar_modelo_e_tokenizer()
        except Exception as e:
            return f"Erro crítico ao carregar o modelo: {e}. Verifique os logs e o caminho do modelo."

    if not pergunta_usuario or not pergunta_usuario.strip():
        return "Por favor, digite uma pergunta válida."


    prefixo_qa = "pergunta: "
    input_text = prefixo_qa + pergunta_usuario.strip()

    logging.info(f"   Texto de entrada para o modelo Q&A: '{input_text}'")
    try:
        # Tokenizar a entrada para o dispositivo correto
        inputs = tokenizer_global(input_text, return_tensors="pt", max_length=512, truncation=True) #.to(device)

        logging.info("   Gerando resposta com modelo Q&A...")
        with torch.no_grad():
            output_ids = model_global.generate(
                inputs["input_ids"],
                max_length=256,          # NOVA ALTERAÇÃO: Alinhado com generation_max_length do fine-tuning Q&A
                num_beams=5,             # Aumentar beams pode melhorar a qualidade, mas torna mais lento
                length_penalty=1.0,      # Neutro. <1.0 para respostas mais curtas, >1.0 para mais longas.
                                         # Para Q&A, respostas concisas e informativas são boas.
                early_stopping=True,     # Importante para parar quando a resposta estiver completa
                no_repeat_ngram_size=3,  # Ajuda a evitar repetições de trigramas

            )

        resposta = tokenizer_global.decode(output_ids[0], skip_special_tokens=True)
        logging.info(f"   Resposta gerada pelo modelo Q&A: '{resposta}'")
        return resposta.strip()

    except Exception as e:
        logging.error(f"❌ ERRO inesperado durante a geração da resposta Q&A: {e}", exc_info=True)
        return f"Desculpe, ocorreu um erro interno ao tentar gerar sua resposta."

# --- Bloco Principal para Interface Gradio ---
if __name__ == '__main__':
    # Tenta carregar o modelo e tokenizer ao iniciar o script
    try:
        logging.info("Iniciando carregamento do modelo Q&A para Gradio...")
        carregar_modelo_e_tokenizer()
    except Exception as e:
        print(f"AVISO INICIAL: Não foi possível carregar o modelo Q&A e tokenizer ao iniciar. Erro: {e}")
        print("A interface Gradio será iniciada, mas a geração de respostas falhará até que o modelo seja carregado com sucesso (ex: corrigindo o caminho e enviando uma pergunta).")

    logging.info(" Lançando interface Gradio para Chatbot de Jardinagem Q&A...")

    interface_description = (
        "Faça uma pergunta sobre como cuidar de plantas, hortas, ou qualquer tema de jardinagem. "
        "Este chatbot usa um modelo Flan-T5-Base ajustado para responder suas dúvidas com base nos dados fornecidos."
    )

    iface = gr.Interface(
        fn=gerar_resposta_chatbot,
        inputs=gr.Textbox(lines=3, placeholder="Ex: Como posso combater pulgões nas minhas roseiras?"),
        outputs=gr.Textbox(label="Resposta do Chatbot de Jardinagem", lines=8),
        title="🌿 Chatbot de Jardinagem Inteligente (Q&A)",
        description=interface_description,
        allow_flagging='never',
        examples=[
            ["Como adubar tomates corretamente?"],
        ]
        # theme=gr.themes.Glass() # Exemplo de tema
    )

    # Lança a interface
    try:
        iface.launch(share=True, debug=False) # debug=False para produção, True para mais logs de Gradio
    except Exception as e:
        logging.error(f"❌ Falha ao lançar a interface Gradio: {e}", exc_info=True)
        print("\n\nSe você estiver no Google Colab e vir um erro sobre 'cannot assign requested address' ou similar,")
        print("tente reiniciar o runtime do Colab e executar novamente, ou tente uma porta diferente com `iface.launch(server_port=7861)`.")
        print("Às vezes, a porta padrão pode estar ocupada ou o Colab pode ter restrições temporárias.")

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://22d399baed24ecdafc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
